# Diffusion 1D

## Question 2(a)

Let's break down the [problem statement](0.Kinetics_Module_Problems.ipynb) for question **2(a)**:

### Declare simulation parameters

**note:**
- Distances are in micrometers
- Diffusion coefficient `DP` is in micrometers squared per second
- Times are in seconds

> A $7~\mathrm{\mu m}$ thick B-doped (p-type) Si wafer is annealed at $950~\mathrm{{}^\circ C}$ while in equilibrium with a gas containing P vapor (donor).

In [ ]:
wafer_thickness = 7.0

> The concentration of B in the wafer is $2\times 10^{17} / \mathrm{cm^3} = 2\times 10^5/\mathrm{\mu m^3}$.

In [ ]:
CB = 2e5 # Bulk concentration of boron

> P diffusivity in this system at this condition is $D_P\approx 10^{-14}~\mathrm{cm^2/s} = 10^{-6}~\mathrm{\mu m^2/s}$.

In [ ]:
DP = 1.0e-6 # Diffusion coefficient

> Assume that the concentration of P at the surface in this case is $10^{21}/\mathrm{cm^3} = 10^9/\mathrm{\mu m^3}$.

In [ ]:
CPsurf = 1.0e9 # Surface concentration of phosphorus

> To fabricate a device with a junction at approximately $1~\mathrm{\mu m}$ from the surface, one would like to match the P and B concentrations at a depth of $1~\mathrm{\mu m}$.

In [ ]:
junction_depth = 1.0 # Distance at which we will output concentration

> Try time steps of 2, 20 and 200 and 2000 s.  For a time of 10 hours and a distance of 1 μm, what value of time step gives agreement with the analytical solution to within one percent?

In [ ]:
dt = 20.0 # Time step
final_time = 36000 # Total amount of time simulated

### Import FiPy

[FiPy](https://www.ctcms.nist.gov/fipy) is an Open Source library for solving partial differential equations, written in Python. We'll use it for the rest of this example (aliased as `fp`).

In [ ]:
import fipy as fp

### Define simulation domain

Subdivide the 1D domain into 700 "cells" of $10~\mathrm{nm}$ width (you can experiment with the effect on the error by changing this value).

In [ ]:
nx = 700 # number of grid elements

mesh = fp.Grid1D(nx=nx, Lx=wafer_thickness)

### Define solution variable(s)

Declare memory storage for the P concentration across the cells of the mesh defined above. Initialize the concentration to zero and inform FiPy that the concentration will evolve in time.
$$C_P(x, t=0) = 0$$

In [ ]:
CP = fp.CellVariable(mesh=mesh, name="$C_P$", value=0.0, hasOld=True)

### Define analytical solution

> The following analytical formula determines the concentration of P at a distance $x = 1~\mathrm{\mu m}$ from the surface to be equal to the concentration of B. 
> 
> \begin{align}
  C_P(x, t) = C_{P,\text{surface}} \left(1 - \mathrm{erf} \frac{x}{2\sqrt{D_P t}}\right)
  \end{align}
> 
> **Note:** The `erf` function and its inverse can be evaluated using SciPy (you can also consider using the complementary error function, `erfc`, which is one minus the error function).

In [ ]:
from scipy.special import erf

def analytical(x, t):
    return CPsurf * (1 - erf(x / (2 * fp.numerix.sqrt(DP * t))))

In [ ]:
CPanalytical = fp.CellVariable(mesh=mesh, name="$C_{P,\text{analytical}}$")
CPanalytical.setValue(analytical(x=mesh.x, t=final_time))

We can also write the above equation as

\begin{align}
  C_P(x, t) = C_{P,\text{surface}} \mathrm{erfc} \frac{x}{2\sqrt{D_P t}}
\end{align}

In [ ]:
from scipy.special import erfc

def analyticalc(x, t):
    return CPsurf * erfc(x / (2 * fp.numerix.sqrt(DP * t)))

What are the advantages of one form over the other? (**Hint:** Try plotting each over a range of $t$ values.)

**note:** It's also possible to import the `sqrt()` function directly from either the `math` or `numpy` package. `fp.numerix` is a superset of `numpy` that customizes some math operations; it's generally best to just use `fp.numerix` everywhere when working with FiPy. 

For comparison:
- [`math`](https://docs.python.org/3/library/math.html) performs calculations with individual numbers.
- [`numpy`](https://numpy.org) performs calculations with arrays of numbers.
- [`fipy.numerix`](https://www.ctcms.nist.gov/fipy/fipy/generated/fipy.tools.html#module-fipy.tools.numerix) performs calculations with fields of numbers that are located on a FiPy `Mesh`.

### Define equation(s)

We want to solve [Fick's 2nd Law of diffusion](https://en.wikipedia.org/wiki/Fick%27s_laws_of_diffusion#Fick's_second_law), the 1D partial differential equation
$$\frac{\partial C_P}{\partial t} = \frac{\partial}{\partial x}\left(D_P \frac{\partial C_P}{\partial x}\right)$$

In [ ]:
eq = fp.TransientTerm(var=CP) == fp.DiffusionTerm(coeff=DP, var=CP)

### Specify boundary condition(s)

\begin{align}
    C_P(x=0, t) &= C_{P,\text{surface}} \\
    \left.\frac{\partial C_P(x, t)}{\partial x}\right|_{x=\mathtt{wafer\underline\ thickness}} &= 0
\end{align}

FiPy's boundary conditions are no-flux by default. We define a [Dirichlet condition](https://en.wikipedia.org/wiki/Dirichlet_boundary_condition) to fix the concentration at the left-hand side of the domain, which we take to be the wafer surface.

In [ ]:
CP.constrain(CPsurf, where=mesh.facesLeft)

### Create viewer

`fp.Viewer()` is a general purpose routine that does its best to create a visualization using a variety of different plotting packages. Explicitly using the [`matplotlib`](https://matplotlib.org) package enables customization for more informative plots.

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
fig, axs = plt.subplots(2, 1, sharex=True)

logviewer = fp.MatplotlibViewer(vars=(CP, CPanalytical),
                                datamin=1e-8, datamax=CPsurf,
                                axes=axs[0], log=True, legend="upper right")
axs[0].set_ylabel(r"$C_p(x,t) / \mathrm{\mu m}^{-3}$")

linviewer = fp.MatplotlibViewer(vars=(CP, CPanalytical),
                                datamin=0., datamax=CPsurf,
                                axes=axs[1], log=False, legend="upper right")
axs[1].set_ylabel(r"$C_p(x,t) / \mathrm{\mu m}^{-3}$")
axs[1].set_xlabel(r"$x / \mathrm{\mu m}$")

plt.tight_layout()

### Solve problem

In [ ]:
# FiPy 3.4.3 has a memory leak when using PETSc
from petsc4py import PETSc

In [ ]:
t = 0
while t <= final_time:
    t=t+dt
    CP.updateOld()
    eq.solve(dt=dt)
    if t % 1800 == 0:
        cm = CP([junction_depth],order=1) # interpolate the value at depth of interest
        ca = analyticalc(x=junction_depth, t=t)
        logviewer.plot()
        linviewer.plot()

        ## for 2.11, uncomment the following lines
        # logviewer.plot(filename=f"{t}-log-plot.png")
        # linviewer.plot(filename=f"{t}-lin-plot.png")
        # fp.TSVViewer(vars=CP).plot(filename=f"{t}-data.tsv")

        print(f"""
        DP = {DP} µm**2/s
        dt = {dt} s
        time = {t} s
        concentration at output distance = {cm} / µm**3
        analytical concentration at output distance = {ca} / µm**3
        error = {100 * (cm - ca) / ca} %
        """)
        
    # FiPy 3.4.3 has a memory leak
    PETSc.garbage_cleanup()